In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split
from torch.optim import AdamW


def load_data(lines):

    print("Loading data...")
    dataset_dict = {}
    for line in lines:
        line = line.strip()

        words = line.split()
        if len(words) > 2: 
            features = words[:-1]
            features = " ".join(features)
            features = features.replace(",", "").replace(".", "")
            label = words[-2:]
            label = " ".join(label)
            label = label.replace(",", "").replace(".", "")
            label = label.strip()
            dataset_dict[features] = label 

    print(dataset_dict)

    return dataset_dict

with open("text.txt", "r") as file:
    lines = file.readlines()



In [14]:

class TextDataset(Dataset):
    def __init__(self, data_dict, tokenizer, max_lenght= 50):

        self.data = list(data_dict.items())
        self.tokenizer = tokenizer
        self.max_length = max_lenght

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        features, label = self.data[idx]
        feature_encoding = self.tokenizer(features, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')['input_ids'].squeeze()
        label_encoding = self.tokenizer(label, truncation=True, padding='max_length', max_length=10, return_tensors='pt')['input_ids'].squeeze()
    
        return feature_encoding, label_encoding
    
def collate_fn(batch):

    feature_encoding = []
    label_encoding = []

    for feature, label in batch:
        feature_encoding.append(feature)
        label_encoding.append(label)

    features = torch.stack(feature_encoding)
    labels = torch.stack(label_encoding)

    return features, labels


In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token 

dataset = TextDataset(load_data(lines), tokenizer)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn, shuffle=True)

Loading data...
{'Before we proceed any further hear me': 'me speak', 'You are all resolved rather to die than to': 'to famish?', 'First you know Caius Marcius is chief enemy to the': 'the people', "We know't we": "we know't", "Let us kill him and we'll have corn at our own": 'own price', "Is't a": 'a verdict?', "No more talking on't; let it be done: away": 'away away!', 'One word good': 'good citizens', 'We are accounted poor citizens the patricians': 'patricians good', 'What authority surfeits on would relieve us: if': 'if they', 'would yield us but the superfluity while it': 'it were', 'wholesome we might guess they relieved us': 'us humanely;', 'but they think we are too dear: the leanness': 'leanness that', 'afflicts us the object of our misery is as': 'as an', 'inventory to particularise their abundance;': 'abundance; our', 'sufferance is a gain to them Let us revenge this': 'this with', 'our pikes ere we become rakes: for the gods know': 'know I', 'speak this in hunger for bread

In [16]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.version.cuda)

class TextGenerationModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, output_size):
        super(TextGenerationModel, self).__init__() 
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_size, batch_first=True, num_layers= 2)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2) 
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)   
        self.fc3 = nn.Linear(hidden_size // 4, output_size) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.emb(x)  
        x, _ = self.lstm(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x) 
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x  
    

model = TextGenerationModel(vocab_size=tokenizer.vocab_size, emb_dim=512, hidden_size=256, output_size=tokenizer.vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = AdamW(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()

    for i , (features, labels) in enumerate(dataloader):
        features, labels = features.to(device), labels.to(device)
        
        outputs = model(features)
        outputs = outputs[:, :labels.size(1), :] 

        outputs = outputs.reshape(-1, outputs.size(-1))  # [batch_size*seq_len, vocab_size]
        labels = labels.reshape(-1)    

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch finished {}/{}. Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

None


KeyboardInterrupt: 

In [ ]:
def eval_encoding(eval_line):
    eval_dict = []

    words = eval_line.split(" ")

    if len(words) > 1:
        features = words
        eval_dict.append(" ".join(features))

    return eval_dict


class InputData():
    def __init__(self, text, tokenizer, max_length=50):
        self.text = text
        self.tokenizer = tokenizer

        self.max_length = max_length

    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        text = self.text[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')['input_ids'].squeeze()

        return {'input_ids': encoding}
    
def eval_collate_fn(batch):

    input_ids = torch.stack([item['input_ids'] for item in batch])
    return input_ids



model.eval()

with open("eval.txt", "r") as file:
    eval_line = file.read()


testdataset = InputData(eval_encoding(eval_line), tokenizer)
test_loader = DataLoader(testdataset, batch_size=32, collate_fn=eval_collate_fn, shuffle=False)

with torch.no_grad():
    for i , (features) in enumerate(test_loader):
        features = features.to(device)
        outputs = model(features)
        class_preds = torch.argmax(outputs, dim=1)
        print(GPT2Tokenizer.from_pretrained('gpt2').convert_ids_to_tokens(class_preds))
        print(class_preds)

['s']
tensor([82], device='cuda:0')
